In [ ]:
%autosave 0
import sys
sys.path.append("../") # go to parent dir
%run si446x_Device_Layer.ipynb
#test

In [ ]:
print('si446x driver version: {}\n'.format(si446x_device_version()))
radio = si446x_device_start_radio()
config = si446x_device_config_radio(radio)
print('compiled config strings (wdds + local:\n{}'.format(config))

In [ ]:
si446x_device_show_config(radio.dump_radio())

## Interactive Inspection of Radio Status

In [ ]:
interact(si446x_device_group_fetch_and_decode, group=radio_config_group_ids.encoding)

In [ ]:
interact(si446x_device_command_fetch_and_decode, cmd=radio_status_cmd_ids.encoding)

## Receive Packets

In [ ]:
def show_interrupts():
    s_name =  'int_status_rsp_s'
    p_s = eval(s_name)
    p_g = radio.get_interrupts()
    p_d = p_s.build(p_g)
    print('{}: {}'.format(s_name, hexlify(p_d)))
    print(radio_display_structs[p_s](p_s, p_d))

In [ ]:
from __future__ import print_function
from datetime import datetime

num          = 500000
MAX_RECV     = 256
MAX_WAIT     = 10
prp          = bytearray(1)
prp[0]        = 25

start = datetime.now()
print(start)

radio.trace._enable()
radio.set_property('PKT', 0x0c, prp)  # set rx fifo threshold

for i in range(1,num):
    msg, rssi, status = si446x_device_receive_msg(radio, MAX_RECV, MAX_WAIT)
#    print(len(msg), rssi)
#    if (status[-1] == 'e'):
    sx = ''
    if (msg):
        m = 16
        x = m if (len(msg) >= m) else len(msg)
        sx += hexlify(msg[:x])
        sx += ".."
        sx += (hexlify(msg[-x:]))
    print('\r{}, c:{}, l:{}, r:{}, t:{}, m:{}, s:{}                  '.format(
        datetime.now(), i, len(msg), rssi, prp[0], sx, ''.join(map(str, status))), end='')
    if (status[-1] == 'e'): print()

show_interrupts()
print('done')

In [ ]:
from datetime import datetime 

now=datetime.now()
print(now)

In [ ]:
reply = radio.dump_trace(0)
print(len(reply))
for tim,where,sig,s_name,dat in reply:
    my_struct = None
    if (s_name == 'string'):
        result = bytearray(dat)
    else:
        result = '{:<20} {}'.format(s_name, binascii.hexlify(bytearray(dat[:8])))
        if ((s_name == 'set_property_cmd_s') or (s_name == 'get_property_cmd_s')
           or (s_name == 'get_property_rsp_s')):
            my_struct = None
        else:
            my_struct = eval(s_name)
    f = '{} {:^16} {}'.format(t.format_time(tim),
                      where,
                      result)
    print(f)
    if (my_struct):
        st = radio_display_structs[my_struct](my_struct, bytearray(dat))
        print('   {}\n'.format(st))

In [ ]:
t = radio.trace
entries = t.filter(filter='RADIO_RSP')
p = t.display(entries)
for entry in p:
    lst = []
    for x in entry[5].split(' '): lst.append(str(x))
    if (not lst):
        s = ''
    elif (lst[1] == 'string'):
        s = ' '.join(lst[2:])
    else:
        s = lst[1]
    print("{} {:<14} {:<20}".format(entry[0],entry[2],entry[5]))
    print(s)